## Web Application Flow

Obtain credentials from your OAuth provider manually. At minimum you will need a client_id but likely also a client_secret. During this process you might also be required to register a default redirect URI to be used by your application. Save these things in your Python script:

In [29]:
from requests_oauthlib import OAuth2Session
from requests import get, post
from urllib import parse
from json import dumps
from pprint import pprint
from IPython.display import display as Display, HTML, Markdown
import os
from pathlib import Path
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = 'true'

In [30]:
client_id = 'ea1544df-3cb1-4b97-8c24-4f69142a2d71'
client_secret = 'my_secret'
redirect_uri = 'http://localhost/my/endpoint'
auth_url ='https://authorization.sandboxcerner.com/tenants/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca/protocols/oauth2/profiles/smart-v1/personas/provider/authorize'
aud='https://fhir-ehr.sandboxcerner.com/r4/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca'
access_token_url = 'https://authorization.sandboxcerner.com/tenants/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca/protocols/oauth2/profiles/smart-v1/token'

### Invoke the authorization workflow using requests_oauthlib OAuth2Session

- set up the GET Authorization code encoded url
- manually fetch the code and status from your web browser
  should return something like this in the browser bar:
  
  `http://localhost/my/endpoint?code=78e3d6c1-9cfa-4a70-a9e9-83bce941ac29&state=7cVNUZbK5WY9m8SyuOTwBrfHfVlh6t`
- enter the code and status in input boxes below...


In [31]:
scope = ["user/CapabilityStatement.read",
            "user/AllergyIntolerance.read",
            "user/Appointment.read",
            "user/Encounter.read",
            "user/Patient.read",
            "user/Practitioner.read",
            "user/RelatedPerson.read",
            "user/Appointment.write",
            "user/Encounter.write",
            "user/Patient.write",]

oauth = OAuth2Session(client_id, redirect_uri=redirect_uri,scope=scope)

authorization_url, state = oauth.authorization_url(auth_url,
                                                  aud=aud)

Display(Markdown(f'**Please go to:** <br />\
                    [this authorization endpoint]({authorization_url}) <br />\
                 **and authorize access.**'))
response_url = input('enter the url response from the browser bar')

**Please go to:** <br />                    [this authorization endpoint](https://authorization.sandboxcerner.com/tenants/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca/protocols/oauth2/profiles/smart-v1/personas/provider/authorize?response_type=code&client_id=ea1544df-3cb1-4b97-8c24-4f69142a2d71&redirect_uri=http%3A%2F%2Flocalhost%2Fmy%2Fendpoint&scope=user%2FCapabilityStatement.read+user%2FAllergyIntolerance.read+user%2FAppointment.read+user%2FEncounter.read+user%2FPatient.read+user%2FPractitioner.read+user%2FRelatedPerson.read+user%2FAppointment.write+user%2FEncounter.write+user%2FPatient.write&state=0rVe3W4D95MSyIUjm6wzIeNBsI4cQK&aud=https%3A%2F%2Ffhir-ehr.sandboxcerner.com%2Fr4%2F0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca) <br />                 **and authorize access.**

enter the url response from the browser bar http://localhost/my/endpoint?code=c758337c-3faf-4fd5-8840-189f0ba65dd8&state=0rVe3W4D95MSyIUjm6wzIeNBsI4cQK


### parse out the authorization code and state from the response url

In [32]:
authorization_code=parse.parse_qs(parse.urlparse(response_url).query)['code'][0]
authorization_code

'c758337c-3faf-4fd5-8840-189f0ba65dd8'

In [33]:
state=parse.parse_qs(parse.urlparse(response_url).query)['state'][0]
state

'0rVe3W4D95MSyIUjm6wzIeNBsI4cQK'

### Get Access token using the authorization code

-  POST a request to the token endpoint by sending these
   parameters using the "application/x-www-form-urlencoded":
   - grant_type='authorization_code',
     - code=authorization_code,
     - client_id=client_id,
     - state=state,
     - redirect_uri = 'http://localhost/my/endpoint',

In [34]:
headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded',
        }
             
body = dict(
        grant_type='authorization_code',
        code=authorization_code,
        client_id=client_id,
        state=state,
        redirect_uri = 'http://localhost/my/endpoint',
            )

r=post(access_token_url,headers=headers,data=body)
print(r.status_code)
print(r.request.url)
for k,v in r.request.headers.items():
    print(f'{k}={v}')
print(r.request.body)    
print(dumps(r.json(), indent=4))


200
https://authorization.sandboxcerner.com/tenants/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca/protocols/oauth2/profiles/smart-v1/token
User-Agent=python-requests/2.21.0
Accept-Encoding=gzip, deflate
Accept=application/json
Connection=keep-alive
Content-Type=application/x-www-form-urlencoded
Content-Length=207
grant_type=authorization_code&code=c758337c-3faf-4fd5-8840-189f0ba65dd8&client_id=ea1544df-3cb1-4b97-8c24-4f69142a2d71&state=0rVe3W4D95MSyIUjm6wzIeNBsI4cQK&redirect_uri=http%3A%2F%2Flocalhost%2Fmy%2Fendpoint
{
    "access_token": "eyJraWQiOiIyMDE5LTA3LTE5VDIzOjQ3OjQwLjY0NC5lYyIsInR5cCI6IkpXVCIsImFsZyI6IkVTMjU2In0.eyJzdWIiOiJwb3J0YWwiLCJ1cm46Y29tOmNlcm5lcjphdXRob3JpemF0aW9uOmNsYWltcyI6eyJ2ZXIiOiIxLjAiLCJ0bnQiOiIwYjhhMDExMS1lOGU2LTRjMjYtYTkxYy01MDY5Y2JjNmIxY2EiLCJhenMiOiJ1c2VyXC9DYXBhYmlsaXR5U3RhdGVtZW50LnJlYWQgdXNlclwvQWxsZXJneUludG9sZXJhbmNlLnJlYWQgdXNlclwvQXBwb2ludG1lbnQucmVhZCB1c2VyXC9FbmNvdW50ZXIucmVhZCB1c2VyXC9QYXRpZW50LnJlYWQgdXNlclwvUHJhY3RpdGlvbmVyLnJlYWQgdXNlclwvUmVsYXRlZFBlcnN

### copy access token to file

In [36]:
p = Path(r'../Validator_Tools/bearer.py')
#print(p)
p.write_text(f"bearer='{r.json()['token_type']} {r.json()['access_token']}'", encoding='utf-8')
print('writing to examples folder......')

writing to examples folder......
